### Load data

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  

Saving vggFeatureExtraction.pt to vggFeatureExtraction (1).pt


In [0]:


!unzip test.zip

Archive:  test.zip
   creating: test/
   creating: test/nonviolent/
  inflating: test/nonviolent/NV_1000.mp42.jpg  
  inflating: test/nonviolent/NV_1000.mp45.jpg  
  inflating: test/nonviolent/NV_1000.mp46.jpg  
  inflating: test/nonviolent/NV_1000.mp48.jpg  
  inflating: test/nonviolent/NV_1000.mp49.jpg  
  inflating: test/nonviolent/NV_18.mp48.jpg  
  inflating: test/nonviolent/NV_18.mp49.jpg  
  inflating: test/nonviolent/NV_19.mp42.jpg  
  inflating: test/nonviolent/NV_19.mp43.jpg  
  inflating: test/nonviolent/NV_19.mp46.jpg  
  inflating: test/nonviolent/NV_191.mp410.jpg  
  inflating: test/nonviolent/NV_191.mp49.jpg  
  inflating: test/nonviolent/NV_192.mp41.jpg  
  inflating: test/nonviolent/NV_192.mp46.jpg  
  inflating: test/nonviolent/NV_192.mp47.jpg  
  inflating: test/nonviolent/NV_193.mp410.jpg  
  inflating: test/nonviolent/NV_193.mp43.jpg  
  inflating: test/nonviolent/NV_193.mp44.jpg  
  inflating: test/nonviolent/NV_193.mp45.jpg  
  inflating: test/nonviolent/NV_206.m

### GradCAM Run


In [0]:
import torch
import torch.nn as nn
from torch.utils import data
from torchvision.models import vgg19
from torchvision.models import vgg11_bn
from torchvision.models import densenet201
from torchvision.models import densenet121
from torchvision import transforms
from torchvision import datasets
import matplotlib.pyplot as plt
import argparse
import numpy as np
import cv2
import gradcam_utils
import os

def fetch_and_fit_model():
    model_ft = models.vgg11_bn(pretrained=True)
    num_ftrs = model_ft.classifier.in_features
    model_ft.classifier = nn.Linear(num_ftrs, 2)
    state = torch.load('../trained_models/densenetFeatureExtraction_l_0.001_m_0.9.pt')
    model_ft.load_state_dict(state['model_state_dict'])
    return model_ft


if __name__ == "__main__":

    parser = argparse.ArgumentParser()
    parser.add_argument('-i', dest='input_directory', action='store', default=os.getcwd(), required=False, help='The root directory containing the image. Use "./" for current directory')
    args = parser.parse_args()

    input_directory = args.input_directory
    transform = transforms.Compose([transforms.Resize((224, 224)), 
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
    dataset = datasets.ImageFolder(root=os.getcwd(), transform=transform)
    dataloader = data.DataLoader(dataset=dataset, shuffle=False, batch_size=1)
    images = []
    for filename in os.listdir(os.path.join(input_directory, 'images')):
        if filename.endswith('.jpg'):
            images.append(filename)

    model_gradcam = fetch_and_fit_model()
    ds, img, scores, label = gradcam_utils.run_inference(model_gradcam, dataloader)
    heatmap = gradcam_utils.get_grad_cam(ds, img, scores, label)
    for image in images:
        gradcam_utils.render_superimposition(input_directory, heatmap, image)
    print(scores)
    print(label)

ModuleNotFoundError: ignored

### GradCAM Util

In [0]:
from torchvision.models import vgg11_bn

model = vgg11_bn(pretrained=True)

print(model)

Downloading: "https://download.pytorch.org/models/vgg11_bn-6002323d.pth" to /root/.cache/torch/checkpoints/vgg11_bn-6002323d.pth
100%|██████████| 507M/507M [00:18<00:00, 28.3MB/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(ke

In [0]:
import torch
import argparse
import torch.nn as nn
from torch.utils import data
from torchvision.models import vgg19
from torchvision.models import densenet201
from torchvision import transforms
from torchvision import datasets
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

class VGG(nn.Module):
    def __init__(self, model):
        # changing because we are using vgg11_bn
        # need to register to the 27t layer RELU before maxpool2d
        super(VGG, self).__init__()
        self.vgg = model
        self.features_conv = self.vgg.features[:28]
        # Use the same params as the pretrained model
        # to verify arch of pretrained print(model)
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2, 
                                     padding=0, dilation=1, ceil_mode=False)
        self.classifier = self.vgg.classifier
        self.gradients = None
    

    def activations_hook(self, grad):
        self.gradients = grad
        

    def forward(self, x):
        x = self.features_conv(x)
        h = x.register_hook(self.activations_hook)
        x = self.max_pool(x)
        x = x.view((1, -1))
        x = self.classifier(x)
        return x
    

    def get_activations_gradient(self):
        return self.gradients
    

    def get_activations(self, x):
        return self.features_conv(x)


def run_inference(model, dataloader):
    vgg = VGG(model)
    vgg.eval()
    img, _ = next(iter(dataloader))
    scores = vgg(img)
    label = torch.argmax(scores)
    return vgg, img, scores, label


def get_grad_cam(vgg, img, scores, label):
  # labels to check the length of tensor
    scores[:, label].backward(retain_graph=True)
    gradients = vgg.get_activations_gradient()
    pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])
    activations = vgg.get_activations(img).detach()
    # 512 is size of conv layer preceding it
    for i in range(512):
        activations[:, i, :, :] *= pooled_gradients[i]
    heatmap = torch.mean(activations, dim=1).squeeze()
    heatmap = np.maximum(heatmap, 0)
    heatmap /= torch.max(heatmap)
    #plt.matshow(heatmap.squeeze())
    return heatmap

def render_superimposition(root_dir, heatmap, image):
    print(os.path.join(root_dir, 'images', image))
    img = cv2.imread(os.path.join(root_dir, 'images', image))
    heatmap = heatmap.numpy()
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = heatmap * 0.4 + img
    cv2.imwrite(root_dir + '/superimposed_' + image, superimposed_img)
    cv2.imshow('output', superimposed_img)